In [1]:
import pandas as pd
url = "datasets/K2P.csv"
k2p_df = pd.read_csv(url, comment='#')
k2p_df.head(5)

,pl_name,hostname,default_flag,disposition,disp_refname,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,...,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,rowupdate,pl_pubdate,releasedate
0,BD+20 594 b,BD+20 594,0,CONFIRMED,Espinoza et al. 2016,1.0,1.0,Transit,2016.0,K2,...,-0.012,9.368,0.018,-0.018,10.8644,0.000249,-0.000249,2018-04-25,2018-03,2018-02-15
1,BD+20 594 b,BD+20 594,0,CONFIRMED,Espinoza et al. 2016,1.0,1.0,Transit,2016.0,K2,...,-0.012,9.368,0.018,-0.018,10.8644,0.000249,-0.000249,2018-04-25,2016-10,2016-07-28
2,BD+20 594 b,BD+20 594,1,CONFIRMED,Espinoza et al. 2016,1.0,1.0,Transit,2016.0,K2,...,-0.012,9.368,0.018,-0.018,10.8644,0.000249,-0.000249,2018-04-25,2017-03,2018-04-26
3,EPIC 201111557.01,EPIC 201111557,0,CANDIDATE,Livingston et al. 2018,1.0,0.0,Transit,2018.0,K2,...,-0.046,9.220,0.019,-0.019,11.3995,0.001307,-0.001307,2018-02-15,2018-03,2018-02-15
4,EPIC 201111557.01,EPIC 201111557,1,CANDIDATE,Livingston et al. 2018,1.0,0.0,Transit,2018.0,K2,...,-0.046,9.220,0.019,-0.019,11.3995,0.001307,-0.001307,2018-08-02,2018-08,2018-08-02


In [2]:
k2p_clean_df = k2p_df.drop(k2p_df.filter(regex="(err1|err2)$").columns, axis=1)
subset = ['pl_name', 'hostname']
k2p_clean_df = k2p_clean_df.drop(subset, axis=1)
k2p_clean_df.isnull().sum().sort_values(ascending=False)

pl_orbeccenlim     3570
pl_orbeccen        3570
pl_bmassprov       3564
pl_bmassjlim       3564
pl_bmasselim       3564
pl_bmasse          3564
pl_bmassj          3564
st_spectype        3559
pl_insol           3374
pl_insollim        3374
pl_orbsmax         3180
pl_orbsmaxlim      3180
pl_eqt             3158
pl_eqtlim          3158
st_metratio        2304
st_metlim          2301
st_met             2301
st_mass            1903
st_masslim         1903
st_logglim         1656
st_logg            1656
st_teff            1126
st_tefflim         1126
pl_rade             843
pl_radjlim          843
pl_radj             843
pl_radelim          843
sy_dist             148
st_radlim           146
st_rad              146
sy_gaiamag           79
pl_orbper            67
pl_orbperlim         67
sy_vmag              65
sy_kmag              46
st_refname           39
decstr               23
ra                   23
rastr                23
ttv_flag             23
dec                  23
rowupdate       

In [3]:
candidates_df = k2p_clean_df[k2p_clean_df["disposition"].str.strip().str.upper() == "CANDIDATE"].copy()

labeled_df = k2p_clean_df[k2p_clean_df["disposition"].str.strip().str.upper().isin(["CONFIRMED","FALSE POSITIVE"])].copy()

labeled_df["label"] = (
    labeled_df["disposition"]
    .str.strip().str.upper()
    .map({"CONFIRMED": 1, "FALSE POSITIVE": 0})
)

labeled_df.drop(["disposition"], axis=1, inplace=True)
labeled_df.head(5)

,default_flag,disp_refname,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,soltype,pl_controv_flag,pl_refname,...,decstr,dec,sy_dist,sy_vmag,sy_kmag,sy_gaiamag,rowupdate,pl_pubdate,releasedate,label
0,0,Espinoza et al. 2016,1.0,1.0,Transit,2016.0,K2,Published Confirmed,0.0,<a refstr=MAYO_ET_AL__2018 href=https://ui.ads...,...,+20d35m56.47s,20.599021,179.461,10.8490,9.368,10.8644,2018-04-25,2018-03,2018-02-15,1
1,0,Espinoza et al. 2016,1.0,1.0,Transit,2016.0,K2,Published Confirmed,0.0,<a refstr=ESPINOZA_ET_AL__2016 href=https://ui...,...,+20d35m56.47s,20.599021,179.461,10.8490,9.368,10.8644,2018-04-25,2016-10,2016-07-28,1
2,1,Espinoza et al. 2016,1.0,1.0,Transit,2016.0,K2,Published Confirmed,0.0,<a refstr=STASSUN_ET_AL__2017 href=https://ui....,...,+20d35m56.47s,20.599021,179.461,10.8490,9.368,10.8644,2018-04-25,2017-03,2018-04-26,1
13,1,del Ser &amp; Fors 2020,1.0,1.0,Transit,2020.0,K2,Published Confirmed,0.0,<a refstr=DEL_SER__AMP__FORS_2020 href=https:/...,...,-04d48m25.21s,-4.807004,NaN,17.6984,12.619,16.4386,2020-11-13,2020-08,2020-11-19,1
21,0,Heller et al. 2019,1.0,1.0,Transit,2019.0,K2,Published Candidate,0.0,<a refstr=KRUSE_ET_AL__2019 href=https://ui.ad...,...,-03d23m22.06s,-3.389461,159.259,16.0140,11.933,15.2612,2019-09-05,2019-09,2019-09-05,1


In [4]:
labeled_df.isnull().sum().sort_values(ascending=False)

st_spectype        2315
pl_orbeccen        2180
pl_orbeccenlim     2180
pl_bmassj          2173
pl_bmasse          2173
pl_bmasselim       2173
pl_bmassjlim       2173
pl_bmassprov       2173
pl_insol           2037
pl_insollim        2037
pl_eqtlim          1852
pl_eqt             1852
pl_orbsmax         1823
pl_orbsmaxlim      1823
st_metratio        1259
st_metlim          1256
st_met             1256
st_mass             888
st_masslim          888
st_logg             880
st_logglim          880
pl_rade             571
pl_radj             571
pl_radelim          571
pl_radjlim          571
st_tefflim          535
st_teff             535
st_radlim           109
st_rad              109
sy_dist              90
sy_vmag              48
pl_orbperlim         47
pl_orbper            47
sy_gaiamag           42
st_refname           39
sy_kmag              37
dec                  23
decstr               23
ttv_flag             23
sy_refname           23
rowupdate            23
pl_pubdate      